# <a id='toc1_'></a>[Building LLM Agents - Red Cross Helpful Information as Aid](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Building LLM Agents - Red Cross Helpful Information as Aid](#toc1_)    
  - [Introduction to LLM Agents](#toc1_1_)    
    - [What Are LLM Agents?](#toc1_1_1_)    
    - [How Do LLM Agents Work?](#toc1_1_2_)    
  - [Building a Basic LLM Agent with OpenAI](#toc1_2_)    
    - [Prompt Engineering](#toc1_2_1_)    
    - [Structured Output](#toc1_2_2_)    
  - [Enhancing LLM Agents with LangChain](#toc1_3_)    
    - [Simple Agent](#toc1_3_1_)    
    - [Web Search Agent](#toc1_3_2_)    
  - [Multi-Agent Systems with LangGraph](#toc1_4_)    
    - [What's a DAG?](#toc1_4_1_)    
      - [Graph](#toc1_4_1_1_)    
      - [Acyclic](#toc1_4_1_2_)    
      - [Directed](#toc1_4_1_3_)    
    - [Where else are DAGs used?](#toc1_4_2_)    
    - [Why Are DAGs Important for LLM Agents?](#toc1_4_3_)    
    - [Building our own DAG - a simple multi-agent system](#toc1_4_4_)    
  - [Putting it all together](#toc1_5_)    
  - [Conclusion](#toc1_6_)    
- [Extra](#toc2_)    
  - [Tool-Using Agent](#toc2_1_)    
  - [RAG Agent](#toc2_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Introduction to LLM Agents](#toc0_)

### <a id='toc1_1_1_'></a>[What Are LLM Agents?](#toc0_)
> Agents are systems that use LLMs as **reasoning engines** to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via tool-calling. *(Source: [LangChain](https://python.langchain.com/docs/tutorials/agents/))*

### <a id='toc1_1_2_'></a>[How Do LLM Agents Work?](#toc0_)
Just like any other LLM. The only difference is that based on the constraints we put on LLM agents (e.g. prompt, structured output), we are able to get them to fulfill a specific role without further model finetuning. An LLM agent typically sits within a pipeline of LLM agents, like we see in [this example](https://langchain-mrkl.streamlit.app/?ref=streamlit-io-gallery-llms). 

## <a id='toc1_2_'></a>[Building a Basic LLM Agent with OpenAI](#toc0_)
Ensure you have the OpenAI Python library installed and use a `.env` file to set up your API key (`OPENAI_API_KEY`).

In [ ]:
# !pip install openai
# !pip install dotenv

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

Set up your API key and define a simple LLM agent function:


In [ ]:
message_history_1 = [{"role": "system", "content": "You are a helpful assistant"}]

In [ ]:
import openai
import os 

def llm_agent(input_, message_history, model="gpt-3.5-turbo"):
    llm = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    message_history.append({"role": "user", "content": f"{input_}"})

    # Generate a response from the chatbot model
    completion = llm.chat.completions.create(
      model=model,
      messages=message_history
    )

    # We save the assistant response
    message_history.append({"role": "assistant", "content": completion.choices[0].message.content})
    return message_history

In [ ]:
llm_agent("I am hungry", message_history_1)

While this is not inherently a bad reply, it doesn't provide any specific support right away. So during this lesson, we'll try to make this better, bit by bit.

### <a id='toc1_2_1_'></a>[Prompt Engineering](#toc0_)

In [ ]:
domain_list = [
    "Shelter",
    "Health & Wellbeing",
    "Safety & Protection",
]

In [ ]:
red_cross_assistant_prompt = """You are an expert query analyzer for a Red Cross virtual assistant.
    Analyze the query to understand the user's needs, emotional state, and language.
    Pay special attention to any signs of emergency or urgent needs.

    If you detect any of these, mark as EMERGENCY:
    - Immediate danger
    - Medical emergencies 
    - Severe distress
    - Threats to basic safety

    If the query is unclear, you should return relevant domains from this list as clarification options:
    {}

    Important: You will analyze:
    1. Query clarity and type (clear/needs_clarification/emergency)
    2. Domains of need, from this list of options {} or "Other"
    3. Emotional state from language and content
    4. Language of query
    """.format(domain_list, domain_list)

In [ ]:
message_history_2 = [{"role": "system", "content": red_cross_assistant_prompt}]

In [ ]:
llm_agent("I am hungry", message_history_2)

Now the issue with this is that it doesn't provide any useful information, it just asks clarifying questions. But that's ok, we can create an LLM with...

### <a id='toc1_2_2_'></a>[Structured Output](#toc0_)

In [ ]:
from typing import List, Literal
from typing_extensions import Union
from pydantic import BaseModel # pydantic is a data validation library widely used in the industry

Domains = Literal[
    "Shelter",
    "Health & Wellbeing",
    "Safety & Protection",
]
DomainWithOther = Union[Domains, Literal["Other"]]

class QueryAnalysis(BaseModel):
    """Analysis output from Query Understanding Agent"""
    query_type: Literal["clear", "needs_clarification", "emergency"]
    domains: List[DomainWithOther]
    emotional_state: str
    language: str 

In [ ]:
def llm_agent(input_, message_history, model="gpt-4o-2024-08-06"):
    llm = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    message_history.append({"role": "user", "content": f"{input_}"})

    # Generate a response from the chatbot model
    completion = llm.beta.chat.completions.parse(
      model=model,
      messages=message_history,
      response_format=QueryAnalysis
    )

    # We save the assistant response
    message_history.append({"role": "assistant", "content": completion.choices[0].message.content})
    return message_history

In [ ]:
llm_agent("I am hungry", message_history_2)

Wonderful! We've already created a useful AI agent. However, if we wanted to switch to a different provider, e.g. Anthropic, or use an open-source LLM, e.g. Llama, DeepSeek, we would need to change our code completely! 

## <a id='toc1_3_'></a>[Enhancing LLM Agents with LangChain](#toc0_)

With LangChain, we can use a lot more models without changing our code each time we change an LLM. Additionally, LangChain is able to couple multiple agents together, allowing an LLM to answer more complex queries more accurately.

In [ ]:
# !pip install langchain
# !pip install langchain_community
# !pip install -U langchain-openai

### <a id='toc1_3_1_'></a>[Simple Agent](#toc0_)

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-3.5-turbo", model_provider="openai")
response = llm.invoke(message_history_1)
response

In [ ]:
response = llm.invoke(message_history_2)
response

In [ ]:
response = llm.with_structured_output(QueryAnalysis).invoke(message_history_2)
response

Now I was able to do the exact same thing with significantly fewer lines of code! Now let's see what other agents we can play with.

### <a id='toc1_3_2_'></a>[Web Search Agent](#toc0_)

In [ ]:
# !pip install -U duckduckgo-search

In [132]:
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

domain_sites = {
        "food": ["voedselbank.nl", "voedselbankennederland.nl"],
        "shelter": ["deregenboog.org", "opvang.nl"],
        "healthcare": ["ggd.nl", "zorgverzekeringslijn.nl"],
        "domestic_violence": ["veiligthuis.nl", "blijfgroep.nl"],
    }

def web_search(query: str) -> dict:
    wrapper = DuckDuckGoSearchAPIWrapper(region="nl-nl", max_results=2) # time='y' limit to past year (m, d, w)
    search_tool = DuckDuckGoSearchResults(api_wrapper=wrapper)
    return search_tool.run(query)

In [133]:
web_search("I am hungry")

'snippet: The expression "Ik heb honger" is a common and neutral way to say "I\'m hungry" in both formal and casual contexts. There are no specific expressions that make the translation formal or casual in this context., title: How do you say "i\'m hungry " in Dutch? | HiNative, link: https://hinative.com/questions/26523530, snippet: Explore the reasons why you might still feel hungry after eating and discover actionable tips to promote fullness and manage hunger effectively., title: Stomach Feel Empty After Eating? 6 Possible Reasons Why | Season, link: https://www.seasonhealth.com/blog/why-does-my-stomach-feel-empty-after-eating, snippet: Waking up hungry is more than just a cue that you\'re ready for breakfast. Three experts on the reasons you\'re waking up hungry and what it says about your health., title: 8 Reasons You May Be Waking Up Hungry, According to Experts - TODAY, link: https://www.today.com/health/health/waking-up-hungry-rcna192814, snippet: Feeling hungry but lacking ap

Awesome! We can search stuff on the internet now. Let's see how we can combine this with our previous agent to get a better search query.

In [ ]:
def perform_web_search(query: str, domains: list = None, location: str = "Netherlands", domain_sites=domain_sites) -> dict:
    """
    Perform a targeted web search with domain-specific site prioritization.
    """
    # Initialize LLM
    llm = ChatAnthropic(
        model="claude-3-5-haiku-20241022",
        temperature=0
    )

    # Prepare relevant sites
    relevant_sites = ["rodekruis.nl"]
    if domains:
        for domain in domains:
            if domain.lower() in domain_sites:
                relevant_sites.extend(domain_sites[domain.lower()])

    # Prepare search query
    system_prompt = """
    Create a simple search query (maximum 10 words) in English or Dutch 
    that will help find relevant assistance information.
    Return ONLY the search query, no explanation or strategy.
    """

    # Generate search query using LLM
    search_query_response = llm.invoke([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"""
            Original query: {query}
            Location: {location}
            Domains: {domains or 'Not specified'}
        """}
    ])

    # Prepare search query with domain priority
    search_query = search_query_response.content
    domain_priority = " OR ".join(f"site:{site}" for site in relevant_sites)
    full_query = f"{search_query} {domain_priority}"

    # Perform web search
    wrapper = DuckDuckGoSearchAPIWrapper(region="nl-nl", max_results=2)
    search_tool = DuckDuckGoSearchResults(api_wrapper=wrapper)
    search_results = search_tool.run(full_query)

    return {
        "query": query,
        "search_query": full_query,
        "results": search_results,
        "domains": domains,
        "location": location
    }

In [ ]:
system_prompt = """
    Create a simple search query (maximum 10 words) in English or Dutch that will find help information.
    Return ONLY the search query, no explanation or strategy.
    """

   

    # Build final search query
    search_query = response.content
    print(search_query)
    domain_priority = " OR ".join(f"site:{site}" for site in relevant_sites)
    full_query = f"{search_query} {domain_priority}"

    # Perform search
    results = web_search(full_query)

    return results

## <a id='toc1_4_'></a>[Multi-Agent Systems with LangGraph](#toc0_)
Although we can already build a multi-agent system with LangChain, we have one constraint: the sequence of actions can only be linear. LangGraph extends LangChain by enabling parallel/conditional interactions through DAGs (Directed Acyclic Graphs).

### <a id='toc1_4_1_'></a>[What's a DAG?](#toc0_)

A **Directed Acyclic Graph (DAG)** is a graph structure where nodes are connected by directed edges, and there are no cycles. This means that information flows in one direction without looping back.

#### <a id='toc1_4_1_1_'></a>[Graph](#toc0_)

A **graph** is a data structure consisting of nodes (or vertices) connected by edges. Graphs are widely used in computer science, mathematics, and AI to model relationships between entities.

![](../../../img/graph_edge_matrix.png)  
(Source: [HuggingFace](http://huggingface.co/blog/intro-graphml))

#### <a id='toc1_4_1_2_'></a>[Acyclic](#toc0_)

- **Cyclic vs. Acyclic Graphs**: A cyclic graph has cycles, whereas an acyclic graph does not.  

![](../../../img/a_cyclical_graphs.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))  

#### <a id='toc1_4_1_3_'></a>[Directed](#toc0_)

**Directed Graphs**: Edges have a specific direction (A → B).  

![](../../../img/directed-graph.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))

### <a id='toc1_4_2_'></a>[Where else are DAGs used?](#toc0_)

![](../../../img/airflow_dag.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))  

DAGs are widely used in streamlining data/ML operations that need to happen sequentially and at regular time intervals. Airflow is one of the most basic platforms where you can build this type of pipelines and monitor the success/failure of your pipelines. You can also schedule runs of pipelines, connect to cloud services, etc. Other schedulers used by data scientists are Prefect (recommended), Dask (for large-scale data processing), Kubeflow pipelines (can be overkill), DAGster. 

You'd do have to do some research before choosing any of these - Airflow is always a good starting point as it's open-source, but users complain that it's outdated compared to competitors.

### <a id='toc1_4_3_'></a>[Why Are DAGs Important for LLM Agents?](#toc0_)
- **Task Execution Flow**: DAGs define clear steps in multi-agent workflows.
- **Parallel Processing**: Independent tasks can run simultaneously.
- **Dependency Management**: Ensures tasks execute in the correct order.

LangGraph uses DAGs to structure multi-agent workflows, ensuring efficient and logical task progression.

### <a id='toc1_4_4_'></a>[Building our own DAG - a simple multi-agent system](#toc0_)

In [ ]:
# !pip install langgraph

In [ ]:
from langgraph.graph import StateGraph
from langgraph.graph.message import Message
from langchain.chat_models import ChatOpenAI




 response = llm.invoke([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"""
            Original query: {query_context['original_query']}
            Location: {query_context['entities'].get('location', 'Netherlands')}
            Domains: {query_context['domains']}
            Language: {query_context['language']}
        """}
    ])

## <a id='toc1_5_'></a>[Putting it all together](#toc0_)

In [ ]:
from langgraph.graph.message import Message
from langchain.chat_models import ChatOpenAI

def query_understanding(state):
    response = ChatOpenAI().predict("Provide an initial analysis on AI safety.")
    return Message(response)

def web_search(state):
    response = ChatOpenAI().predict(f"Expand on the following: {state['message']}")
    return Message(response)

def emergency_node(state):
    """Returns emergency contact information"""
    whatsapp_number = "ADD RED CROSS WHATSAPP # HERE"
    return {
        "messages": [
            {
                "role": "assistant",
                "content": f"""  
                    This seems urgent and like you need immediate assistance. Please contact the Red Cross directly at this number {whatsapp_number}  
                    to get help immediately.  
                    For any medical emergency please contact 112.  
                    """
            }
        ]
    }

In [ ]:
from langgraph.graph import StateGraph

def route_by_query_type(state):
    """
    Routes to appropriate node based on query analysis
    In query_understanding QueryAnalysis.query_type Literal["clear", "needs_clarification", "emergency"]
    """
    analysis = state["analysis"]
    # Route based on query type
    if analysis["query_type"] == "clear":
        return "web_agent"
    elif analysis["query_type"] == "emergency":
        return "emergency"
        
workflow = StateGraph()
workflow.add_node("await_clarification", await_clarification_node)
workflow.add_node("emergency", emergency_node)
workflow.add_node("web_agent", web_agent_node)
workflow.add_edge(START, "query_understanding")
workflow.add_conditional_edges(
        "query_understanding",
        route_by_query_type,
        {
            "web_agent": "web_agent",
            "emergency": "emergency",
        }
    )
workflow.add_edge("emergency", END)
workflow.add_edge("web_agent", END)
workflow.run()

In [ ]:
workflow.run()

## <a id='toc1_6_'></a>[Conclusion](#toc0_)
- **LLM agents** enable intelligent text-based automation.
- **LangChain** enhances LLMs with tools, memory, and structured pipelines.
- **LangGraph** facilitates multi-agent workflows for complex AI interactions.

With these tools, you can build powerful AI-driven applications. 🚀

# <a id='toc2_'></a>[Extra](#toc0_)

## <a id='toc2_1_'></a>[Tool-Using Agent](#toc0_)
An agent that integrates with APIs or tools (e.g., calculator, weather API, web scraper).

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain_core.tools import tool

@tool # When we use the scraping tool, the output will be compatible with LangChain
def web_scrape_tool(url):
    """Sends simple request to websites and retrieves the answer"""
    try:
        response = requests.get(url)
        content = BeautifulSoup(response.text, 'html.parser')
        content_list = [p.text.replace('\n', '') for p in content.find_all('p')]
        final_content = "\n".join(content_list)
        return final_content[:1000]  # Limit to first 1000 characters
    except Exception as e:
        return f"Error scraping {url}: {str(e)}"

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

example_url = "https://en.wikipedia.org/wiki/LangChain"
message_history = [
    AIMessage("You are a web research assistant who can scrape and summarize web content."),
    HumanMessage(f"Can you summarize the info on this webpage: {example_url}?"),
]
llm = init_chat_model("gpt-3.5-turbo", model_provider="openai", temperature=0).bind_tools([web_scrape_tool])
response = llm.invoke(message_history)
response

In [ ]:
for tool_call in response.tool_calls:
    if tool_call['name'].lower() == 'web_scrape_tool':
        url = tool_call['args']['url']
        tool_output = web_scrape_tool(url)
        tool_message = ToolMessage(
            content=tool_output, 
            tool_call_id=tool_call['id']
        )
        message_history.append(response)
        message_history.append(tool_message)

message_history

In [ ]:
llm.invoke(message_history)

As you can see, we didn't get an answer to our query. Instead, we were able to extract the URL from the user's prompt. To also get the result, we need to pass the tool input as a message!

## <a id='toc2_2_'></a>[RAG Agent](#toc0_)

RAG Agents require a bit more in-depth view over vector databases and embeddings but you can find a tutorial [here](https://www.kdnuggets.com/implement-agentic-rag-using-langchain-part-2).